experimento GA.01 - senha de tamanho variavel
========================================



## Introdução



Delete este texto e escreva uma breve introdução sobre seu experimento.



## Objetivo



**Objetivo**: Resolver o problema da senha sem fornecer a informação do tamanho da senha para a função que gera a população.

**Dica**: A função objetivo terá que quantificar em sua métrica tanto se o candidato acertou as letras quanto se acertou o tamanho da senha.

## Importações



Todos os comandos de `import` devem estar dentro desta seção.



In [1]:
from funcoes import populacao_inicial_senha_variavel
from funcoes import funcao_objetivo_pop_senha_variavel
from funcoes import selecao_torneio_min
from funcoes import cruzamento_ponto_senha_variavel as funcao_cruzamento
from funcoes import mutacao_senha_variavel_tamanho
from funcoes import mutacao_senha
from funcoes import gene_letra
import random

## Códigos e discussão



In [2]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 10
NUM_GERACOES = 1000
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.5
NUM_COMBATENTES_NO_TORNEIO = 2

# relacionadas ao problema a ser resulvido
MAXIMO = 100
PESO_PENALIDADE = 60
SENHA = "isalindadmsgatona" # o que estou tentando descobrir
LETRAS_POSSIVEIS = "abcdefghijklmnopqrstuvwxyz" # todas letras que podem estar na minha senha
# NUM_GENES = len(SENHA)

In [3]:
# funções locais

def cria_populacao_inicial(tamanho, tamanho_senha):
    return populacao_inicial_senha_variavel(TAMANHO_POP, MAXIMO, LETRAS_POSSIVEIS) #usa um tamanho máximo chute 

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_senha_variavel(populacao, SENHA , PESO_PENALIDADE)

def funcao_selecao(populacao, fitness):
    return selecao_torneio_min(populacao, fitness, tamanho_torneio=6)

#def funcao_mutacao(individuo):
#    return mutacao_senha(individuo, LETRAS_POSSIVEIS)

def funcao_mutacao(individuo):
    return mutacao_senha_variavel_tamanho(individuo, LETRAS_POSSIVEIS)

In [4]:
populacao = populacao_inicial_senha_variavel(TAMANHO_POP, MAXIMO, LETRAS_POSSIVEIS)

melhor_fitness_ja_visto = float("inf")  # é assim que escrevemos infinito em python
                        # assim, o primeiro indivíduo já substituirá, 
                        # pois vai ser melhor que infinito

print("Progresso da melhor senha já vista:")

while melhor_fitness_ja_visto != 0:
    
    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)
    
    # Cruzamento
    pais = populacao[0::2] # os pais vão pares (em númros cardinais)
    maes = populacao[1::2] # as mães serão ímpares
    
    contador = 0
    
    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2
            
        contador = contador + 2   
        
    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)
            

    # melhor individuo já visto até agora
    fitness = funcao_objetivo_pop(populacao)
    menor_fitness = min(fitness)
    
    if menor_fitness < melhor_fitness_ja_visto:        
        posicao = fitness.index(menor_fitness)
        melhor_individuo_ja_visto = populacao[posicao]
        melhor_fitness_ja_visto = menor_fitness
        
        print("".join(melhor_individuo_ja_visto), "- fitness:", melhor_fitness_ja_visto)

print()
print("Melhor palpite da senha encontrado:")
print("".join(melhor_individuo_ja_visto))

Progresso da melhor senha já vista:
dwnjsskskhjtfljliy - fitness: 186
dwnjsskskhjkfljliy - fitness: 177
dwnjqskskhjkfljliy - fitness: 175
dwnjmskskhjkfljliy - fitness: 171
dwnjmskskhjkfljli - fitness: 111
dwnjjskskhskfljli - fitness: 99
dwnjjsknkhskfljli - fitness: 94
dwnjjsknkiskfljli - fitness: 93
dwijjsknkhskfljli - fitness: 89
dwijjsbnkhskfljli - fitness: 84
dwimjsbnkhskfljli - fitness: 83
dsimjsbnkhukfljli - fitness: 81
dsimjsbnkhukfrjli - fitness: 75
dsimjsbnkhukfrjld - fitness: 70
dsimjsbgkhukfrjld - fitness: 63
dsimjnbnkhukfrmld - fitness: 62
dsimjnbnkhukfsmld - fitness: 61
dsimjnbnkhujfsmld - fitness: 60
dsimjnbekhujfsmld - fitness: 51
jzgljngeenshfthrh - fitness: 48
jzgljngeenshfthoh - fitness: 45
jzgljngeenshfthnh - fitness: 44
jzgljneeenshfthnh - fitness: 42
jzgljneeenshfthnd - fitness: 38
jzaljneeenshfthnh - fitness: 36
jwaljneeensjfthnh - fitness: 35
jzaljneeenshfttnh - fitness: 34
jpaljneeenujftmnh - fitness: 31
jpaljneeensjftmnh - fitness: 29
jpaljneeensgftmnh - fitness

## Conclusão



Eu usei o mesmo escopo das funções para o código de descobrir senha, mudei que ao invés de chutar uma senha com o mesmo tamanho, eu defini um número máximo de letras que ele poderia chutar, então tive que criar uma função mutação que arrumava tanto o número de letras quanto quais letras eram.
Não precisei mudar nada no algoritmo genético pois escolhi fazer uma única função de mutação que fazia os dois ajustes ao mesmo tempo, porém se eu quisese duas funções de mutação eu teria que mudar o algoritmo.
Esse código é muito lento, demora muito para descobrir a senha e eu não consegui fazer com que ele descobrisse senhas muito grandes, testei com no máximo 17 letras, ainda sim tive que ajustar a função objetivo para atribuir uma distância quadrática quanto ao tamanho da senha, se não ele não diminuia.

## Playground



Todo código de teste que não faz parte do seu experimento deve vir aqui. Este código não será considerado na avaliação.



In [5]:
caracteres = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "z"]

for caractere in caracteres:
    numero = ord(caractere)
    print(f"O caractere '{caractere}' converte para o número {numero}")

O caractere 'a' converte para o número 97
O caractere 'b' converte para o número 98
O caractere 'c' converte para o número 99
O caractere 'd' converte para o número 100
O caractere 'e' converte para o número 101
O caractere 'f' converte para o número 102
O caractere 'g' converte para o número 103
O caractere 'h' converte para o número 104
O caractere 'i' converte para o número 105
O caractere 'z' converte para o número 122


In [6]:
individuo = ['a','b','c','d']
gene = random.randint(0, len(individuo) - 1)
individuo.insert(gene, gene_letra(LETRAS_POSSIVEIS))
print(individuo)

['s', 'a', 'b', 'c', 'd']
